In [2]:
# Import libraries
import spacy
import langdetect
from tqdm.notebook import tqdm
import pdfplumber
import fitz
import os
import re
import sys
import traceback
import pickle

In [3]:
# Define headings in the document using word size function (PDFPlumber)
def word_ratio_func(word):
    try:
        # calculate word size parameters
        word_length = len(word[4])
        word_bottom = float(word[1])
        word_top = float(word[3])
        return abs(word_bottom - word_top), word_length, word[4]
        
    except:
        # in case of error, return zeros
        return 0, 0, 0

In [4]:
# Define headings in the document using word size function (Fitz)
def word_ratio_func(word):
    try:
        # calculate word size parameters
        word_length = len(word["text"])
        word_bottom = float(word['bottom'])
        word_top = float(word['top'])
        return (word_bottom - word_top), word_length, word["text"]
        
    except:
        # in case of error, return zeros
        return 0, 0, 0

In [5]:
# Preprocess the text
def preprocess_text(texts):

    # join the text and perform cleansing operations
    text = "".join(texts.values()).strip("●").strip("*")
    text = text.split("\n")
    text = [x for x in text if x != '' and x.startswith("Source") == False]
    text = [x[0].replace("●", "") + x[1:] if x[0] == "●" else x for x in text]
    text = [x[0].replace("1", "") + x[1:] if x[1:3] in ["Q1", "Q2", "Q3", "Q4"] else x for x in text]
    text = text[2:]

    return text

In [6]:
# Define if table is correctly identified by calculating digit to character ratio
def digit_character_ratio(s):
    digit_count = 0
    char_count = 0

    for char in s:
        if char.isdigit():
            digit_count += 1
        if char.isalpha() or char.isdigit():
            char_count += 1

    # Avoid division by zero
    if char_count == 0:
        return 0

    return digit_count / char_count

In [7]:
from difflib import SequenceMatcher
# Define similarity function
def similar(a, b):
    return SequenceMatcher(None, a, b).ratio()

### PDF Into Texts

In [8]:
# Transform pdf into the text
def process_pdf(pdf_paths):
    
    # store the pdf text and headings in dictionaries
    pdf_texts = {}
    pdf_headings = {}

    # iterate over the pdf files
    for file_path in tqdm(pdf_paths):

        # file_path = "ShareholderLetters/Investor-Letter-Q1-2012.pdf"
        # file_path = "ShareholderLetters/Investor-Letter-Q3-2011.pdf"
        # file_path = "ShareholderLetters/July-Investor-Letter-1130am.pdf"
        # file_path = "ShareholderLetters/July2014EarningsLetter_7.21.14_final.pdf"
        # file_path = "ShareholderLetters/Q4_14_Letter_to_shareholders.pdf"
        # file_path = "ShareholderLetters/FINAL-Q1-23-Shareholder-Letter.pdf"
        # file_path = "ShareholderLetters/FINAL-Q2-23-Shareholder-Letter.pdf"

        try:

            # store the text and headings within one pdf file
            texts = {}
            headings = []
            headings_count = 0
            fitz_flag = False

            # open the pdf file using pdfplumber library
            plumber_reader = pdfplumber.open(file_path)

            # for page_number in range(len(fitz_reader)):
            # iterate over pages in the pdf document
            for page_number in range(0, len(plumber_reader.pages)):

                # get the specific page from the pdf plumber
                plumber_page = plumber_reader.pages[page_number]
                # get text from page using pdf plumber
                text = plumber_page.extract_text()

                # if text is not correctly extracted and spaced using pdf plumber
                if len(text.split(" ")) / len(text) < 0.15 or fitz_flag:
                    # set fitz library flag as true for further pages
                    fitz_flag = True
                    # open the pdf file using fitz library
                    fitz_reader = fitz.open(file_path)
                    # get the specific page from the fitz library
                    fitz_page = fitz_reader.load_page(page_number)
                    # get text from page using fitz library
                    text = fitz_page.get_text()

                # find table from the page
                table = plumber_page.extract_tables()
                # if table exists on the page
                if len(table):
                    # define the beginning of table
                    start_table = table[0][0][0].split("\n")[0]
                    # define the end of table
                    end_table = table[-1][-1]
                    end_table = [x for x in end_table if x is not None][-1]
                    # if table has non-empty start and ending
                    if start_table != '' and end_table != '':
                        # flatten the table list of words and digits
                        table_list = [item for sublist in table for subsublist in sublist for item in subsublist]
                        # remove None values from the list
                        table_list = [x for x in table_list if x is not None]
                        # join the list as single string
                        table_list = " ".join(table_list)
                        # calculate the digit to character ratio
                        ratio = digit_character_ratio(table_list)
                        # remove table if digit to character ratio is over 0.2, meaning that table contains numeric data
                        if ratio > 0.2:
                            # update the page text by removing table
                            text = text.split(start_table)[0] + text.split(end_table)[-1]

                # add text to dictionary of texts
                texts[page_number] = text

                # words = page.get_text("words")
                ### get headings from page
                # extract words
                words = plumber_page.extract_words()
                word_count = 0
                # iterate over words
                while word_count < len(words):
                    # find if the words are large enough to be headings by calculating their size
                    word_size, word_length, word_text = word_ratio_func(words[word_count])
                    heading = []

                    # if word size is over 13.5, this means that the word is heading
                    if word_size > 13.5 and word_length > 1:
                        # append the following words if they satisfy this heading size condition
                        while True:
                            heading.append(word_text)
                            word_count += 1
                            if word_count >= len(words):
                                break
                            word_size, word_length, word_text = word_ratio_func(words[word_count])
                            # if word is small again, break the loop and finish the heading
                            if not word_size > 13.5 and word_length > 1:
                                headings.append(" ".join(heading))
                                # add the indent of 10 words to avoid issues 
                                word_count += 10
                                break
                    headings_count += 1
                    word_count += 1
        
                # break if the page covers the reference section
                if "Reference" in heading or page_number == 10:
                    break
            
            # preprocess the text
            text = preprocess_text(texts)
            final_text = " ".join(text).strip()

            

            # optionally, export the text to a txt file
            # with open("Txt/" + file_path.split("/")[-1].split(".")[0] + ".txt", "w", encoding='utf-8') as f:
            #     f.write(final_text)

            # add the text to the dictionary
            pdf_texts[file_path.split("/")[-1].split(".")[0]] = final_text
            # clean the headings
            headings = [x.replace("\u200b", "remove") for x in headings]
            headings = [x for x in headings if not re.search("remove", x)]    
            # add the headings to the dictionary
            pdf_headings[file_path.split("/")[-1].split(".")[0]] = headings

            # break

        except Exception as e:

            # in case of error, print the specifics of issue
            exc_type, exc_value, exc_traceback = sys.exc_info()
            traceback_details = traceback.extract_tb(exc_traceback)
            filename = traceback_details[-1][0]
            line_no = traceback_details[-1][1]
            func = traceback_details[-1][2]
            print(f"Exception occurred in file {filename} at line {line_no} in function {func}")
            print(f"Exception type: {exc_type.__name__}, Exception message: {str(e)}")

            # break

            # continue
    
    return pdf_texts, pdf_headings

In [ ]:
# Get file paths for the pdf files
folder_path = "ShareholderLetters/" # put '/' sign at the end of the folder
file_paths = []
for root, directories, files in os.walk(folder_path):
    for filename in files:
        filepath = os.path.join(root, filename)
        file_paths.append(filepath)

# Transform pdf files into texts and headings and store them as dictionaries
pdf_texts, pdf_headings = process_pdf(file_paths) # total run time: 2 min 20 s 20 files

In [79]:
# Save pdf texts and headings to pickle files
# with open("pdf_texts2.pkl", "wb") as f:
#     pickle.dump(pdf_texts, f)
# with open("pdf_headings2.pkl", "wb") as f:
#     pickle.dump(pdf_headings, f)

In [27]:
# file_path = "ShareholderLetters/Investor_Letter_Q12013.pdf"
# file_path = "ShareholderLetters/FINAL-Q3-22-Shareholder-Letter.pdf"
# file_path = "ShareholderLetters/Investor-Letter-Q3-2011.pdf"
# file_path = "ShareholderLetters/July-Investor-Letter-1130am.pdf"
# file_path = "ShareholderLetters/July2014EarningsLetter_7.21.14_final.pdf"
# file_path = "ShareholderLetters/Q3_14_Letter_to_shareholders.pdf"
# file_path = "ShareholderLetters/Q4_14_Letter_to_shareholders.pdf"
# file_path = "ShareholderLetters/FINAL-Q2-23-Shareholder-Letter.pdf"
# file_path = "ShareholderLetters/FINAL-Q1-23-Shareholder-Letter.pdf"
file_path = "ShareholderLetters/Investor-Letter-Q3-2011.pdf"
# file_path = "ShareholderLetters/Investor-Letter-Q42012-01.pdf"

page_number = 0

pdf_plumber = pdfplumber.open(file_path)
plumber_page = pdf_plumber.pages[page_number]

fitz_reader = fitz.open(file_path)
fitz_page = fitz_reader.load_page(page_number)
text = fitz_page.get_text()

plumber_text = plumber_page.extract_text()
table = plumber_page.extract_tables()

if len(plumber_page.extract_tables()):
    print("Table found")
    start_table = plumber_page.extract_tables()[0][0][0].split("\n")[0]
    if not re.search(start_table, text):
        start_table = start_table.replace(" ", "")
    end_table = plumber_page.extract_tables()[-1][-1]
    end_table = [x for x in end_table if x is not None]
    end_table = end_table[-1]
    if start_table != '' and end_table != '':
        table_list = plumber_page.extract_tables()
        table_list = [item for sublist in table_list for subsublist in sublist for item in subsublist]
        table_list = [x for x in table_list if x is not None]
        table_list = " ".join(table_list)
        ratio = digit_character_ratio(table_list)
        print(ratio)
        if ratio > 0.2:
            text_2 = text.split(start_table)[0] + text.split(end_table)[-1]

Table found
0.5042918454935622


### Pdfreader is not good

### Extracting Tables

In [31]:
pdf_document = "ShareholderLetters/Investor-Letter-Q1-2012.pdf"
pdf_document = "ShareholderLetters/FINAL-Q3-22-Shareholder-Letter.pdf"

pdf_plumber = pdfplumber.open(pdf_document)
page = pdf_plumber.pages[11]
table = page.extract_tables()
if len(table):
    print("Table found")
    start_table = table[0][0][0]
    end_table = table[0][-1][-1]

Table found


In [32]:
table

[[['Revenues $ 7,925,589 $ 7,970,141 $ 7,483,467 $ 23,763,497 $ 21,988,526',
   None,
   None,
   None,
   None,
   None,
   None,
   None,
   None],
  ['Cost of revenues 4,788,665 4,690,755 4,206,589 13,764,125 12,093,108',
   None,
   None,
   None,
   None,
   None,
   None,
   None,
   None],
  ['Marketing 567,954 574,960 635,948 1,698,892 1,752,433',
   None,
   None,
   None,
   None,
   None,
   None,
   None,
   None],
  ['Technology and development 662,739 716,846 563,887 2,037,115 1,626,415',
   None,
   None,
   None,
   None,
   None,
   None,
   None,
   None],
  ['General and administrative 373,213 409,297 321,790 1,180,438 953,831',
   None,
   None,
   None,
   None,
   None,
   None,
   None,
   None],
  ['Operating income 1,533,018 1,578,283 1,755,253 5,082,927 5,562,739',
   None,
   None,
   None,
   None,
   None,
   None,
   None,
   None],
  ['Other income (expense):', None, None, None, None, None, None, None, None],
  ['Interest expense (172,575) (175,455) (190,

In [33]:
import pandas as pd

# Convert the list into a DataFrame
df = pd.DataFrame(table[1:], columns=table[0])

# Display the DataFrame
df.head(100)

,"Revenues $ 7,925,589 $ 7,970,141 $ 7,483,467 $ 23,763,497 $ 21,988,526",NaN
,"Cost of revenues 4,788,665 4,690,755 4,206,589 13,764,125 12,093,108",NaN
,"Marketing 567,954 574,960 635,948 1,698,892 1,752,433",NaN
,"Technology and development 662,739 716,846 563,887 2,037,115 1,626,415",NaN
,"General and administrative 373,213 409,297 321,790 1,180,438 953,831",NaN
,"Operating income 1,533,018 1,578,283 1,755,253 5,082,927 5,562,739",NaN
,Other income (expense):,NaN
,"Interest expense (172,575) (175,455) (190,429) (535,609) (576,191)",NaN
,"Interest and other income 261,404 220,226 96,135 677,275 302,702",NaN
,"Income before income taxes 1,621,847 1,623,054 1,660,959 5,224,593 5,289,250",NaN
,"Provision for income taxes (223,605) (182,103) (211,888) (787,953) (780,451)",NaN


In [56]:
import pdfplumber

pdf_document = "ShareholderLetters/Investor-Letter-Q1-2012.pdf"
page_number = 0  # Replace with the page number you're interested in

with pdfplumber.open(pdf_document) as pdf:
    page = pdf.pages[page_number]
    
    # Extract tables as DataFrame objects
    tables = page.extract_tables()
    
    # Process the tables
    for table in tables:
        for row in table:
            print(row)
        print("-" * 20)

['(in millions except per share data)', "Q2 '10", "Q3 '10", "Q4 '10", "Q1 '11", "Q2 '11", "Q3 '11", "Q4 '11", "Q1 '12"]
['Domestic Streaming:', '', '', '', '', '', '', '', 'G id']
['Net Subscription Additions', '-', '-', '-', '-', '-', '-', '0.22', '1.74']
['Total Subscriptions', '-', '-', '-', '-', '-', '21.45', '21.67', '23.41']
['Paid Subscriptions', '-', '-', '-', '-', '-', '20.51', '20.15', '22.02']
['Revenue', '-', '-', '-', '-', '-', '-', '$ 476', '$ 507']
['Contribution Profit', '-', '-', '-', '-', '-', '-', '$ 52', '$ 67']
['Contribution Margin', '-', '-', '-', '-', '-', '-', '10.9%', '13.2%']
['', '', '', '', '', '', '', '', '']
['International Streaming:', '', '', '', '', '', '', '', '']
['Net Subscription Additions', '-', '0.13', '0.38', '0.29', '0.16', '0.51', '0.38', '1.21']
['Total Subscriptions', '-', '0.13', '0.51', '0.80', '0.97', '1.48', '1.86', '3.07']
['Paid Subscriptions', '-', '-', '0.33', '0.67', '0.86', '0.99', '1.45', '2.41']
['Revenue', '-', '$ -', '$ 4', '$ 

In [36]:
text

" \n \n1 \nApril 23rd, 2012    \nDear Fellow Shareholders, \nNetflix added nearly 3 million streaming members in Q1, bringing our total to over 26 million global \nstreaming members, and strengthening our position as the world’s leading Internet TV network.   We \nanticipate returning to global profitability in Q2, and plan to launch our next international market in Q4.   \nWe are constantly improving our service with better personalization, better user-interfaces, better \nstreaming, and more content.  As a result, per-member viewing hours set new records in Q1 and are on \ntrack to do so again in Q2, on a year-over-year basis.  We launched our service in the UK and Ireland in \nJanuary and are very pleased that, after the first 90 days, we had substantially more members than we \nhad after the first 90 days of Canada or Latin America.       \n \n (in millions except per share data)\nQ2 '10\nQ3 '10\nQ4 '10\nQ1 '11\nQ2 '11\nQ3 '11\nQ4 '11\nQ1 '12 \nG id\n \nDomestic Streaming:\nNet Sub

In [34]:
import re
import fitz

pdf_document = "ShareholderLetters/Investor-Letter-Q1-2012.pdf"
page_number = 0  # Replace with the page number you're interested in

doc = fitz.open(pdf_document)
page = doc[page_number]

text = page.get_text()

# Define regular expressions for tabular data
row_pattern = re.compile(r'\n.*\n')  # Match rows based on newline
column_pattern = re.compile(r'\s+')  # Match columns based on whitespace

rows = row_pattern.findall(text)
table_detected = False

# Check if tabular data is detected
if len(rows) > 1:
    columns = column_pattern.split(rows[0])
    if len(columns) > 1:
        table_detected = True

if table_detected:
    print("Table detected on the page.")
else:
    print("No table detected on the page.")

Table detected on the page.


### Cleaning the data

In [2]:
# Load the language model
nlp = spacy.load("en_core_web_sm")

In [4]:
# Create a function to clean the data
def clean_data(df):
    # Create a dictionary to store the values
    new_df = {"label": [], "text": []}

    # Iterate over all rows in the dataset
    for row in tqdm(range(len(df.loc[:, "text"].to_list()))):
        # Initialize temporary array to store tokens
        tmp_tokens = []

        try:
            # # Check whether the review is written in English or not
            # if langdetect.detect(df.loc[row, "text"]) == "en" or True:
                for token in nlp(df.loc[row, "text"]):
                    # Set conditions to retain valuable information
                    if (
                        not token.is_stop  # remove stop-words
                        and not token.is_punct  # remove punctuation
                        and not token.like_num  # remove numbers
                        and token.is_oov  # remove words that don't have a word vector
                        and not token.is_space  # remove whitespaces
                        and len(token) > 1  # remove single-letter words
                        # Remove tokens that looks weird & not useful
                        and not str(token).endswith("-")
                        and not str(token).endswith(".")
                        and not any(
                            substr in str(token)
                            for substr in [
                                "---",
                                "--",
                                "/2",
                                "/1",
                                "20feb",
                                "c17",
                                "\x92",
                                "&",
                                "%",
                                "i.e.",
                                "b+",
                                "w/",
                                "02:33:05",
                            ]
                        )
                        and not str(token).startswith("-")
                    ):
                        # Get the lemma & lowercase the token
                        token = token.lemma_.lower()
                        if "(" in token:
                            token = token.split("(")
                            tmp_tokens.append(token[0])
                            tmp_tokens.append(token[1])
                        elif token == "orangy/":
                            token = "orangy"
                        elif token == ".fruity":
                            token = "fruity"

                        tmp_tokens.append(token)

                # Append the corresponding label to the review
                new_df["label"].append(df.loc[row, "label"])

                # Add all tokens from the review to the text
                new_df["text"].append(tmp_tokens)
                # Reset the token array
                tmp_tokens = []
        except:
            continue  # proceed to next row if an exception is raised
        
    # Return the new dataframe
    return pd.DataFrame(new_df)